# Решение задачи определения эмоциональной окраски отзывов к фильмам с IMDB с помощью нескольких способов

1) Сначала данные загружаются из Keras, немного преобразуются и тренируется модель

2) Потом загружаются предобученные векторные представления, а данные загружаются архивом и вручную разбиваются на папки. Модель тренируется с использованием предобученных представлений

3) Затем модель тренируется на тех же данных, разбитых на папки, но уже без 
предварительно обученных представлений

Скачивание данных

In [ ]:
from keras.datasets import imdb


# всего слов 10_000
max_words = 10_000
# из каждого отзыва будем брать только первые 20 слов
maxlength = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)
x_train

Предобработка данных

In [ ]:
from keras import preprocessing

# преобразование массива целых чисел в двумерный тензор формы (образцы, максимальная длина)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlength)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlength)
x_train

Создание модели

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
# Определение максимальной длины входа для слоя Embedding в 
# целях последующего уменьшения размерности.
# После слоя Embedding активация имеет форму (образцы, максимальная_длина, 8)
# первый аргумент - размер словаря (количество слов)
# второй аргумент - одна из размерностей выходного тензора (снижение размерности)
# eсли после Embedding есть Flatten или Dense, то необходимо указывать input_length
model.add(Embedding(10_000, 8, input_length=maxlength))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Тренировка модели

In [ ]:
history = model.fit(
    x_train,
    y_train, 
    batch_size=32,
    validation_split=0.2, 
    epochs = 30
)
history = history.history

Функции для рисование графиков потерь и точности

In [ ]:
import matplotlib.pyplot as plt

def draw_loss(history):
    loss_values = history["loss"]

    epochs = range(1, len(history['loss']) + 1)
    plt.plot(epochs, loss_values, 'b', label='Training loss')
    plt.title('Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    plt.show()
    plt.clf()

def draw_acc(history):
    acc_values = history["accuracy"]

    epochs = range(1, len(history['accuracy']) + 1)
    plt.plot(epochs, acc_values, 'r', label='Training accuracy')
    plt.title('Training accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

draw_loss(history)
draw_acc(history)

# Теперь будем использовать предарительно обученное векторное представление слов, но данные IMDB загрузим не из Keras, а архивом и распределим вручную

In [ ]:
import os

data_dir = os.path.join('/content/drive/My Drive/StudyingKeras/IMDB/data/aclImdb.zip (Unzipped Files)/aclImdb/train')

# массив меток
labels = []
# массив строк - одна строка на отзыв
texts = []

# метки двух типов: положительные и отрицательные
for label_type in ['neg', 'pos']:
  # по-очереди переходи между папками
  dir_name = os.path.join(data_dir, label_type)
  print(f"Processing file in {dir_name}")
  print(dir_name)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      with open(os.path.join(dir_name, fname)) as f:
        texts.append(f.read())
      if label_type == 'neg':
        labels.append(0)
      elif label_type == 'pos':
        labels.append(1)

Токенизация текста из исходного набора данных IMDB

In [ ]:
# поскольку мы используем предварительно обученные векторные представления, они не 
# всегда идеально подходят для любой задачи
# стоит ограничить тренировочные данные, чтобы снизить объем вариантов фраз
# возьмем первые 200 образцов

import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# сто положительных и сто отрицательных отзывов
maxlen = 100
# обучение на выборке из 200 образцов
training_samples = 200
# проверка на выборке из 10 000 образцов
validation_samples = 10_000
# рассматриаем только 10 000 самых частых слов
max_words = 10_000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique sequences.")

# преобразование массива целых чисел в двумерный тензор формы (образцы, максимальная длина)
data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print(f'Shape of data tensor is {data.shape}')
print(f'Shape of labels tensor is {labels.shape}')

# перед тем как разбить данные на тренировочную и тестовую выборки, их 
# необходимо перемешать, так как изначально они упорядочены
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# выборки по 200 образцов
x_train = data[:training_samples]
y_train = labels[:training_samples]
# на валидацию данные берем из общих тренировочных
x_val = data[training_samples : training_samples + validation_samples]
y_val = labels[training_samples : training_samples + validation_samples]



Разделение данных их файла на тренировочные и тестовые занимает слишком много времени. Они буду сохранены в файл для последующей быстрой работы с ними.

In [ ]:
import pickle

with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/x_train', 'wb') as f:
  pickle.dump(x_train, f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/y_train', 'wb') as f:
  pickle.dump(y_train, f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/x_val', 'wb') as f:
  pickle.dump(x_val, f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/y_val', 'wb') as f:
  pickle.dump(y_val, f)


Обработка файла с векторными представлениями слов GloVe

In [ ]:
glove_dir = '/content/drive/My Drive/StudyingKeras/IMDB/GloVe weights/glove.6B.zip (Unzipped Files)'
embedding_index = {}

with open(os.path.join(glove_dir, 'glove.6B.100d.txt')) as f:
  for line in f:
    values = line.split()
    # первое значение - само слово
    word = values[0]
    # остальные значения - коэффициента для построение вектора
    coefs = np.asarray(values[1:], dtype='float32')
    # все помещаем в наш словарь
    embedding_index[word] = coefs

print(f"Found {len(embedding_index)} pre-trained word vectors")

Found 400000 pre-trained word vectors


Создание матрицы векторных представлений слов GloVe

In [ ]:
# это должна быть матрица формы (число слов, размерность представления)
# каждый элемент матрицы содержит вектор такой же размерности, что и вектор
# i-ого слова в созданном словаре embedding_index

embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in  word_index.items():
  if i < max_words:
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

# Создание модели с предварительно обученным векторным представлением слов

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

pretrained_model = Sequential()
pretrained_model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
pretrained_model.add(Flatten())
pretrained_model.add(Dense(32, activation='relu'))
pretrained_model.add(Dense(1, activation='sigmoid'))
pretrained_model.summary()

Загрузка предварительно обученных векторных представлений слов в слой Embedding

In [ ]:
# загружаем веса в слой (новый метод)
pretrained_model.layers[0].set_weights([embedding_matrix])
# замораживаем слой, чтобы веса не изменялись
pretrained_model.layers[0].trainable = False
pretrained_model.summary()

Компиляция и обучение модели

In [ ]:
pretrained_model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
pretrained_history = pretrained_model.fit(
    x_train,
    y_train, 
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
).history
pretrained_model.save('/content/drive/My Drive/StudyingKeras/IMDB/weights/pre_trained_glove_model.h5')

Создание графиков обучения

In [ ]:
draw_loss(pretrained_history)
draw_acc(pretrained_history)

# Создание модели без предварительно обученного векторного представления слов

In [ ]:
raw_model = Sequential()
# сколько слов максимум, глубина вектора слова, длина входных слов
raw_model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
raw_model.add(Flatten())
raw_model.add(Dense(32, activation='relu'))
raw_model.add(Dense(1, activation='sigmoid'))
raw_model.summary()

Компиляция и обучение модели

In [ ]:
raw_model.compile(
    optimizer='rmsprop', 
    loss='binary_crossentropy',
    metrics=['accuracy']
)
raw_history = raw_model.fit(
    x_train, 
    y_train,
    batch_size=32,
    epochs=10,
    validation_data=(x_val, y_val)
).history
raw_model.save('/content/drive/My Drive/StudyingKeras/IMDB/weights/raw_model.h5')

Создание графиков обучения

In [ ]:
draw_loss(raw_history)
draw_acc(raw_history)

# мы видим, что в данном случае точность на проверке хуже
# это значит, что при небольшом объеме выборки (200 слов),
# предварительно обученное представление выигрывает у вновь
# обученного
# если же увеличить размер выборки, то ситуация резко изменится 
# на противоположную

Токенизация контрольных данных

In [ ]:
data_dir = os.path.join('/content/drive/My Drive/StudyingKeras/IMDB/data/aclImdb.zip (Unzipped Files)/aclImdb/test')

# массив меток
labels = []
# массив строк - одна строка на отзыв
texts = []

# метки двух типов: положительные и отрицательные
for label_type in ['neg', 'pos']:
  # по-очереди переходи между папками
  dir_name = os.path.join(data_dir, label_type)
  print(dir_name)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      with open(os.path.join(dir_name, fname)) as f:
        texts.append(f.read())
      if label_type == 'neg':
        labels.append(0)
      elif label_type == 'pos':
        labels.append(1)
    
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

/content/drive/My Drive/StudyingKeras/IMDB/data/aclImdb.zip (Unzipped Files)/aclImdb/test/neg
/content/drive/My Drive/StudyingKeras/IMDB/data/aclImdb.zip (Unzipped Files)/aclImdb/test/pos


Предыдущий шаг занимает очень много времени, так что сохраним полученные переменные в файлы

In [ ]:
import pickle

with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/test/x_test', 'wb') as f:
  pickle.dump(x_test, f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/test/y_test', 'wb') as f:
  pickle.dump(y_test, f)

Загрузка токенизированных данных из pickle файлов

In [ ]:
import pickle
import numpy as np

maxlen = 100
max_words = 10_000
embedding_dim = 100

with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/test/x_test', 'rb') as f:
  x_test = pickle.load(f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/test/y_test', 'rb') as f:
  y_test = pickle.load(f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/x_train', 'rb') as f:
  x_train = pickle.load(f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/y_train', 'rb') as f:
  y_train = pickle.load(f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/x_val', 'rb') as f:
  x_val = pickle.load(f)
with open('/content/drive/My Drive/StudyingKeras/IMDB/pickled/train/y_val', 'rb') as f:
  y_val = pickle.load(f)


# Оценка моделей

Оценка модели c предобучением на вручную собранном контрольном наборе данных

In [ ]:
from keras.models import load_model

pretrained_model = load_model('/content/drive/My Drive/StudyingKeras/IMDB/weights/pre_trained_glove_model.h5')
pretrained_model.evaluate(x_test, y_test)

Оценка заново натренированной модели на вручную собранном контрольном наборе данных

In [ ]:
from keras.models import load_model

raw_model = load_model('/content/drive/My Drive/StudyingKeras/IMDB/weights/raw_model.h5')
raw_model.evaluate(x_test, y_test)